In [1]:
import geopandas as gpd
import pandas as pd
from utils import gdf_operations, functions

2025-01-29 15:44:32.342 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
distrito =gdf_operations.get_dados('distrito')
subbac = gdf_operations.get_dados('subbac')
subpref = gdf_operations.get_dados('subpref')
fcu = gdf_operations.get_dados('fcu')

In [3]:
cd_subbac = gdf_operations.find_subbac_name(subbac, 'cd_')
nm_subbac = gdf_operations.find_subbac_name(subbac, 'nm_')

unidades_list = [
    ('subbac', "Sub Bacias Hidrográficas", cd_subbac, nm_subbac),
    ('subpref', "Subprefeituras", "cd_subpref" , 'nm_subpref'),
    ('distrito', "Distritos", 'cd_distrit', 'nm_distrit'),
    ('fcu', "Favelas e Comunidades Urbanas", 'cd_fcu', 'nm_fcu')
    
]
unidades_df = pd.DataFrame(unidades_list, columns=['gdf_name', 'name', 'column_cd', 'column_name'])


Vou começar pelo fcu para ver se dá certo.

# FCU

In [4]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [5]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

gdf_name
fcu
column_name


overlay_gdf = gdf_operations.overlay_intersec(
    'distrito', gdf_unidade, 'cd_fcu')

# Distritos

Agora vamos tentar com os distritos

In [6]:
distrito_sorted = gdf_operations.create_gdf_sorted(
    distrito, 'distrito'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [7]:
choice_unidade = 'distrito'
gdf_unidade = distrito_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

gdf_name
distrito
column_name


In [8]:
overlay_gdf = gdf_operations.overlay_intersec(
    'subpref', gdf_unidade, 'cd_distrit'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


# Loop por intersec

## Vamos Começar com o FCU mesmo

In [9]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [10]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

gdf_name
fcu
column_name


In [11]:
print(unidades_df[unidades_df['gdf_name']=='fcu'])

  gdf_name                           name column_cd column_name
3      fcu  Favelas e Comunidades Urbanas    cd_fcu      nm_fcu


In [12]:
index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == choice_unidade
        ].index[0]
    )

name_gdf_unidade = choice_unidade
batatinha = pd.DataFrame()

cd_unidade = gdf_operations.find_gdf_name(gdf_unidade, choice_unidade, 'cd_')

overlay_gdf = gpd.GeoDataFrame()


for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )
            print(overlay_gdf.columns)
            
            gdf_unidade[cd_unidade] = gdf_unidade[cd_unidade].astype(str)
            overlay_gdf[cd_intersec] =(
                overlay_gdf[cd_intersec]
                .astype(int)
                .astype(str)
            )
            
            overlay_gdf[cd_unidade] = (
                overlay_gdf[cd_unidade]
                .astype(str)
            )


            gdf_unidade = pd.merge(gdf_unidade, overlay_gdf[[cd_unidade, cd_intersec]], on=cd_unidade, how='left')


gdf_operations.save_intersec(gdf_unidade, choice_unidade)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'nm_bacia_h', 'geometry', 'area'],
      dtype='object')


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'cd_subpref', 'nm_subpref', 'geometry', 'area'],
      dtype='object')


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'cd_subpref', 'cd_distrit', 'nm_distrit', 'geometry', 'area'],
      dtype='object')


In [13]:
gdf_unidade.sample(3)

,cd_fcu,nm_fcu,pop_fcu,dom_fcu,con_fcu,geometry,cd_identif,cd_subpref,cd_distrit
744,35503081168,Parque Pereira,547,182,76580,"POLYGON ((344286.268 7387686.776, 344224.278 7...",592,29,72
642,35503080502,Jardim Arnaldo,1415,496,198100,"POLYGON ((322136.082 7379085.589, 322104.167 7...",563,18,46
1278,35503081881,Jardim Bonifácio,151,51,21140,"MULTIPOLYGON (((354051.046 7391673.208, 354073...",706,27,47


## Intersec distrito

In [14]:
distrito_sorted = gdf_operations.create_gdf_sorted(
    distrito, 'distrito'
)
choice_unidade = 'distrito'
gdf_unidade = distrito_sorted
name_column_unidade = functions.find_gdf_info(
    unidades_df, 
    choice_unidade, 
    'gdf_name', 
    'column_name'
)

gdf_name
distrito
column_name


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [15]:
index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == choice_unidade
        ].index[0]
    )

name_gdf_unidade = choice_unidade
batatinha = pd.DataFrame()

cd_unidade = gdf_operations.find_gdf_name(gdf_unidade, choice_unidade, 'cd_')

overlay_gdf = gpd.GeoDataFrame()


for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )
            print(overlay_gdf.columns)
            
            gdf_unidade[cd_unidade] = gdf_unidade[cd_unidade].astype(str)
            overlay_gdf[cd_intersec] = overlay_gdf[cd_intersec].astype(int).astype(str)
            overlay_gdf[cd_unidade] = overlay_gdf[cd_unidade].astype(str)


            gdf_unidade = pd.merge(gdf_unidade, overlay_gdf[[cd_unidade, cd_intersec]], on=cd_unidade, how='left')


gdf_operations.save_intersec(gdf_unidade, choice_unidade)


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_distrit', 'nm_distrit', 'pop_total', 'dom_total', 'consumo_di',
       'cd_identif', 'nm_bacia_h', 'geometry', 'area'],
      dtype='object')


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_distrit', 'nm_distrit', 'pop_total', 'dom_total', 'consumo_di',
       'cd_identif', 'cd_subpref', 'nm_subpref', 'geometry', 'area'],
      dtype='object')


In [16]:
gdf_unidade.sample(3)

,cd_distrit,nm_distrit,pop_total,dom_total,consumo_di,geometry,cd_identif,cd_subpref
29,53,MOOCA,80901,38380,11326140,"POLYGON ((338349.356 7394264.944, 338362.239 7...",631,25
62,94,VILA SONIA,123299,53632,17261860,"POLYGON ((320805.525 7390280.495, 320824.353 7...",620,10
75,17,CAMPO LIMPO,236766,98086,33147240,"POLYGON ((321265.375 7388006.57, 321292.309 73...",620,17


## Testar DE NOVO O LOOP

In [17]:
def intersec_unidades(
    unidades_df,
    gdf_unidade:gpd.GeoDataFrame,
    name_gdf_unidade:str
    
):
    gdf_sorted = gdf_operations.create_gdf_sorted(
        gdf_unidade, 
        name_gdf_unidade
    )

    gdf_unidade= gdf_sorted

    name_column_unidade = gdf_operations.find_gdf_name(
        gdf_unidade, name_gdf_unidade, 'nm_'
    )
#    name_column_unidade = functions.find_gdf_info(unidades_df, name_gdf_unidade, 'gdf_name', 'column_name')

    index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == name_gdf_unidade
        ].index[0]
    )

  

    cd_unidade = gdf_operations.find_gdf_name(
        gdf_unidade, 
        name_gdf_unidade, 
        'cd_'
    )

    overlay_gdf = gpd.GeoDataFrame()

    for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )

            
            gdf_unidade[cd_unidade] = (
                gdf_unidade[cd_unidade].astype(str)
            )
            overlay_gdf[cd_intersec] = (
                overlay_gdf[cd_intersec]
                .astype(int)
                .astype(str)
            )
            overlay_gdf[cd_unidade] = (
                overlay_gdf[cd_unidade].astype(str)
            )

    


            gdf_unidade = pd.merge(
                gdf_unidade, 
                overlay_gdf[[cd_unidade, cd_intersec]], 
                on=cd_unidade, 
                how='left'
            )

    
    gdf_operations.save_intersec(gdf_unidade, choice_unidade)

    return gdf_unidade
            

In [18]:
intersec_fcu = intersec_unidades(unidades_df, fcu, 'fcu')
intersec_fcu.sample(3)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]
D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geo

,cd_fcu,nm_fcu,pop_fcu,dom_fcu,con_fcu,geometry,cd_identif,cd_subpref,cd_distrit
26,35503080578,Parque Santo Antônio V,256,92,35840,"POLYGON ((321094.953 7382357.707, 321096.151 7...",581,18,46
1194,35503080376,Jardim Iporanga / Esmeralda,6873,2561,962220,"POLYGON ((326454.957 7374056.218, 326446.723 7...",550,19,23
401,35503080749,Domênico Lansete,3057,1103,427980,"POLYGON ((326428.829 7364148.308, 326421.311 7...",540,20,55


In [19]:
intersec_distrito = intersec_unidades(unidades_df, distrito, 'distrito')
intersec_distrito.sample(3)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]
D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geo

,cd_distrit,nm_distrit,pop_total,dom_total,consumo_di,geometry,cd_identif,cd_subpref
91,8,BELEM,55551,25348,7777140,"POLYGON ((336956.857 7397163.748, 336962.822 7...",646,25
38,62,PINHEIROS,65782,37900,9209480,"POLYGON ((326414.868 7392091.012, 326411.759 7...",633,11
39,63,PIRITUBA,179512,78262,25131680,"POLYGON ((325109.668 7405122.828, 325106.786 7...",726,2


In [20]:
intersec_subpref = intersec_unidades(
    unidades_df, subpref, 'subpref')
intersec_subpref.sample(3)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]
D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geo

,cd_subpref,nm_subpref,pop_total,dom_total,consumo_di,geometry,cd_identif
17,18,M BOI MIRIM,571456,231712,80003840,"POLYGON ((319648.289 7371582.185, 319417.128 7...",580
7,28,GUAIANASES,273768,107011,38327520,"POLYGON ((354736.042 7396035.315, 354700.624 7...",706
19,20,PARELHEIROS,164954,68177,23093560,"POLYGON ((319189.103 7359417.595, 319188.984 7...",504


In [21]:
intersec_subbac = intersec_unidades(
    unidades_df, subbac, 'subbac')
intersec_subbac.sample(3)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


,cd_identif,nm_bacia_h,pop_total,dom_total,consumo_di,geometry
150,643.0,Córrego Bussocaba,111223,47928,15571220,"POLYGON ((318049.563 7396728.699, 318132.901 7..."
248,738.0,Córrego Cabuçu de Baixo,397658,159695,55672120,"POLYGON ((328620.266 7409536.706, 328655.324 7..."
13,513.0,Ribeirão Grande,693,327,97020,"POLYGON ((320818.62 7363052.825, 320830.793 73..."


Por algum motivo só salva os shapes dos distritos e fcu